In [0]:
def mount_adls_by_extra_app(
    tenant_id: str,
    client_id: str,
    client_secret: str,
    storage_account_name: str,
    container_name: str,
    replace_if_exists: bool = False
):
    configs = {
        "fs.azure.account.auth.type": "OAuth",
        "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
        "fs.azure.account.oauth2.client.id": client_id,
        "fs.azure.account.oauth2.client.secret": client_secret,
        "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
    }
    
    # Unmount the mount point if it already exists
    if any(
        mount.mountPoint == f"/mnt/{storage_account_name}/{container_name}"
        for mount in dbutils.fs.mounts()
    ):
        if not replace_if_exists:
            display(dbutils.fs.mounts())
            return
        dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")
    
    # Mount the storage account container
    dbutils.fs.mount(
        source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
        mount_point = f"/mnt/{storage_account_name}/{container_name}",
        extra_configs = configs)

    display(dbutils.fs.mounts())


client_id = dbutils.secrets.get(scope = "formula1-scope", key = "formula1-app-client-id")
tenant_id = dbutils.secrets.get(scope = "formula1-scope", key = "formula1-app-tenant-id")
client_secret = dbutils.secrets.get(scope = "formula1-scope", key = "formula1-app-client-secret")

In [0]:
dbutils.secrets.help()

Provides utilities for leveraging secrets within notebooks.
Databricks documentation for more info.
 get(scope: String, key: String): String -> Gets the string representation of a secret value with scope and key getBytes(scope: String, key: String): byte[] -> Gets the bytes representation of a secret value with scope and key list(scope: String): Seq -> Lists secret metadata for secrets within a scope listScopes: Seq -> Lists secret scopes

In [0]:
dbutils.secrets.listScopes()

[SecretScope(name='explore-databricks'),
 SecretScope(name='explore-databricks-2'),
 SecretScope(name='explore-databricks-3'),
 SecretScope(name='kvexploredatabricks')]

In [0]:
dbutils.secrets.list("kvexploredatabricks")

[SecretMetadata(key='app-explore-databricks-clientid'),
 SecretMetadata(key='app-explore-databricks-secret'),
 SecretMetadata(key='tenant-id')]

In [0]:
dbutils.secrets.get("kvexploredatabricks", "tenant-id")

'[REDACTED]'